# Lennard Jones - NVE ensemble

Tutorial for a simple MD simulation of Lennard-Jones spheres operating under the microcanonical ensemble.

NOTE: This tutorial has been adapted from the LJ tutorial located at https://bitbucket.org/glotzer/hoomd-examples.

NOTE: A detailed description of all HOOMD commands can be found at https://hoomd-blue.readthedocs.io/en/stable/.

## Initialize

Import the hoomd python package and the md component to execute MD simulations.

In [ ]:
import hoomd
import hoomd.md

Initialize the execution context to control where HOOMD will execute the simulation. When no command line options are provided, HOOMD will auto-select a GPU if it exists, or run on the CPU.

In [ ]:
hoomd.context.initialize("")

Initialize an $n$ by $n$ by $n$ simple cubic lattice of particles, where `a` represents the lattice constant. The lattice initializer by default creates all particles named type "A", and with 0 velocity.

In [ ]:
lattice = hoomd.init.create_lattice(unitcell=hoomd.lattice.sc(a=2.0), n=5)

Initialize particle velocities from a Gaussian distribution.

In [ ]:
import random
random.seed(1)
T_init = 0.1
for p in lattice.particles:
    p.velocity = (random.gauss(0, T_init), random.gauss(0, T_init), random.gauss(0, T_init))

## Define potential energy

$ V(r) = 4 \varepsilon \left[ \left( \frac{\sigma}{r} \right)^{12} - \left( \frac{\sigma}{r} \right)^{6} \right] $, where $r$ < $r$<sub>cut</sub>

In the Lennard-Jones system, pairs of particles closer than $r_\mathrm{cut}$ interact with this potential energy.

Choose the neighbor list acceleration structure to find neighboring particles efficiently. In systems with only one cutoff length, the cell method performs best.

In [ ]:
nl = hoomd.md.nlist.cell(r_buff=0.6, check_period=1)

Define the functional form of the pair interaction and evaluate using the given neighbor list acceleration structure.

In [ ]:
lj = hoomd.md.pair.lj(r_cut=2.5, nlist=nl)

Specify pair potential parameters for every pair of types in the simulation.

In [ ]:
lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0)

## Select integrator

The integrator defines the equations of motion that govern the system of particles, given the current configuration of the particles and the net force from all potentials. The standard integration mode in HOOMD allows different integrators to apply to different groups of particles with the same step size $dt$.

In [ ]:
hoomd.md.integrate.mode_standard(dt=0.005)

Apply NVE integration using the Velocity-Verlet algorithm.

In [ ]:
all = hoomd.group.all()
hoomd.md.integrate.nve(group=all)

## Write output

The `hoomd.analyze.log` method can be used to log a variety of system properties (see http://hoomd-blue.readthedocs.io/en/stable/module-hoomd-analyze.html#hoomd.analyze.log). Here we will periodically log the temperature of the system to a text file.

In [ ]:
hoomd.analyze.log(filename="analyze.log",
                  quantities=['temperature'],
                  period=100,
                  overwrite=True)

Periodically write the particle configurations to a gsd file.

In [ ]:
hoomd.dump.gsd("trajectory.gsd", period=2e4, group=all, overwrite=True)

## Run the simulation

Take 1,000,000 steps forward in time.

In [ ]:
hoomd.run(1e6)

## Examine the output

Use matplotlib to plot the potential energy vs time step.

In [ ]:
import numpy
from matplotlib import pyplot
%matplotlib inline
data = numpy.genfromtxt(fname='analyze.log', skip_header=True);

In [ ]:
pyplot.figure(figsize=(4,2.2), dpi=140);
pyplot.plot(data[:,0], data[:,1]);
pyplot.xlabel('time step');
pyplot.ylabel('temperature');

Examine how the system configuration evolves over time. [ex_render](ex_render.py) is a helper script that builds animated gifs from trajectory files and system snapshots. It is part of the [hoomd-examples](https://bitbucket.org/glotzer/hoomd-examples) repository and designed only to render these examples.

In [ ]:
import ex_render
ex_render.display_movie(ex_render.render_sphere_frame, 'trajectory.gsd');